# Lag STAC fra GeoTiff

### Input: 
Mappe med .tif og .tfw -filer. Filer kan lastes ned fra Geonorge her: [(N50 Raster (UTM32) - Rutevis)](https://kartkatalog.geonorge.no/metadata/n50-raster-utm32-rutevis/bbe517ba-3b3a-4c96-a78e-104f6c4804d1?search=n50)

### Output:
**/stac_catalog** som inneholder *catalog.json* (oversikt) og egen mappe for hvert innleste kartblad.

## Import av bibliotek 

In [32]:
import os
import rasterio
import numpy as np
import utils
import shutil
import subprocess
from pathlib import Path
from pystac import Catalog, Item, Asset
from datetime import datetime
from shapely.geometry import box, mapping

## Mappestruktur
Her er det tenkt at koden kjøres i en mappe som har en underkatalog som heter "Input". Den skal inneholde de originale Tiff-filene.
Det vil bli oppretta to andre mapper: 

* **"COG"**: Her lagres konverterte COG-er.  
* **"Output"**: Her opprettes STAC-katalogen (/stac_catalogue)

Følgende kode sjekker først at **Input**-mappa eksisterer. Hvis det er tilfelle sjekkes det om det eksisterer **COG**- og **Output**-mapper. Hvis de finnes tømmes innholdet, hvis ikke de finnes opprettes de. Da vil mappa se slik ut:

N50TilSTAC/  
├─ Input/  
├─ COG/    
├─ Output/  
└─ (andre filer, som f.eks. .gitignore))

In [33]:
# Definer input/output mapper
inputFolder = utils.get_workdir() / "Input"

if inputFolder.exists():
    # Sjekk om output-mappe eksisterer
    outputFolder = utils.get_workdir() / "Output"
    if outputFolder.exists():
        for item in outputFolder.iterdir():
            if item.is_file():
                item.unlink()
            elif item.is_dir():
                shutil.rmtree(item)
    else:
        # Opprett COG-mappe hvis den ikke finnes
        outputFolder.mkdir(parents=True, exist_ok=True)

    # Sjekk om COG-mappe eksisterer
    COGFolder = utils.get_workdir() / "COG"
    if COGFolder.exists():
        for item in COGFolder.iterdir():
            if item.is_file():
                item.unlink()
            elif item.is_dir():
                shutil.rmtree(item)
    else:
        # Opprett COG-mappe hvis den ikke finnes
        COGFolder.mkdir(parents=True, exist_ok=True)

    catalog = Catalog(id="N50-test-catalog", description="STAC Catalog for N50 maps")
else:
    print("Input-mappa eksisterer ikke!")
    exit()

## UDF: Konverter GeoTiff til COG

Funksjonen konverterer alle .tif-filer i Input-mappa til COG filer i/COG-mappa.
  
Resultatet (COG-filene) lagres i "N50TilSTAS/COG/".

In [34]:
def convert_folder_to_cog(input_folder, COGFolder):
    input_folder = Path(input_folder)
    COGFolder = Path(COGFolder)
    COGFolder.mkdir(parents=True, exist_ok=True)

    # Tellere
    nTellerAntTiffFiler = 0 # Antall TIFF-filer
    nTellerAntBehandla = 0 # Antall behandlede filer

    # Gå gjennom alle .tif-filer i input-mappen
    for tif_file in input_folder.glob("*.tif"):
        # Legg til "_cog" før filendelsen
        output_file = COGFolder / f"{tif_file.stem}_cog{tif_file.suffix}"
        print(f"Konverterer {tif_file.name} til {output_file.name}...")

        # Tell antall filer som skal behandles
        nTellerAntTiffFiler += 1

        try:
            # Åpne input-TIFF
            with rasterio.open(tif_file) as src:
                data: np.ndarray = src.read()       # shape: (bands, height, width)
                profile: dict = src.profile.copy()

                # Oppdater profil for COG
                profile.update({
                    "driver": "COG",
                    "compress": "DEFLATE",              # tilsvarer -co COMPRESS=DEFLATE
                    "TILING_SCHEME": "GoogleMapsCompatible",  # tilsvarer -co TILING_SCHEME=GoogleMapsCompatible
                    "nodata": 0
                })

                # Lag maske for NODATA-håndtering
                mask = np.zeros(src.shape[1:], dtype=int)

                # Rødt bånd - NODATA: 255
                mask = np.where(data[0] == 255, mask + 1, mask)

                # Grønt bånd - NODATA: 255
                mask = np.where(data[1] == 255, mask + 1, mask)

                # Blått bånd - NODATA: 252
                mask = np.where(data[2] == 252, mask + 1, mask)

                # Endre alle verdier som er 0 til 1
                data[data == 0] = 1

                # Dersom alle bånd hadde sin respektive NODATA, settes verdien til 0
                data = np.where(mask == 3, 0, data)

            # Skriv ut COG
            with rasterio.open(output_file, "w", **profile) as dst:
                dst.write(data)

            # Tell antall behandla filer
            nTellerAntBehandla += 1

        except Exception as e:
            print(f"Konvertering mislyktes for {tif_file.name}: {e}")

## Les og legg filene til i STAC-katalogen

In [ ]:
# Her konverteres filene
convert_folder_to_cog(inputFolder, COGFolder)

nTellerAntCogFiler = 0

for filename in os.listdir(COGFolder):
    if filename.endswith(".tif"): # Velg kun TIFF-filer (COG-filer har også .tif som endelse)
        filepath = COGFolder / filename

        # Les bbox og EPSG-kode fra TIFF-filen
        with rasterio.open(filepath) as src:
            # Hent bounding box for fila
            bounds = src.bounds
            bbox = [bounds.left, bounds.bottom, bounds.right, bounds.top]
            geometry = mapping(box(*bbox)) # Omriss
            crs = src.crs.to_string() # EPSG-kode

        item_id = os.path.splitext(filename)[0]
        item = Item(
            id=item_id,
            geometry=geometry,
            bbox=bbox,
            datetime=datetime.now(),
            properties={"proj:epsg": int(crs.split(":")[-1])},
        )

        # Output-katalog
        item_output_dir = outputFolder / "stac_catalog" / item_id
        item_output_dir.mkdir(parents=True, exist_ok=True)
        item_path = item_output_dir / f"{item_id}.json"

        # Relativ sti til bildet
        rel_image_href = os.path.relpath(filepath, start=item_path.parent)
        item.add_asset("image", Asset(href=rel_image_href, media_type="image/tiff"))

        # Hvis tfw-hjelpefiler finnes legges de til som "georef"
        twf_path = inputFolder / f"{item_id}.tfw"
        if twf_path.exists():
            rel_tfw_href = os.path.relpath(twf_path, start=item_path.parent)
            item.add_asset(
                "georef",
                Asset(
                    href=rel_tfw_href,
                    media_type="text/plain",
                    roles=["auxiliary", "georeferencing"],
                ),
            )

        nTellerAntCogFiler += 1

        catalog.add_item(item)

# Sjekk om alle .cog-filer er behandla
if nTellerAntCogFiler == 0:
    print("Input-mappa er tom!")
else:
    # Lagre katalogen
    catalog.normalize_and_save(
    root_href=str(outputFolder / "stac_catalog"),
    catalog_type="SELF_CONTAINED"
)



Konverterer 32_N50raster_2471.tif til 32_N50raster_2471_cog.tif...
Konverterer 32_N50raster_2472.tif til 32_N50raster_2472_cog.tif...
Konverterer 32_N50raster_2473.tif til 32_N50raster_2473_cog.tif...
